# MPlot how-to
This notebook is a tutorial for the use of the methods defined in nbs\mplots.ipybn for analyzig MPlots. This tool is introduced by Eamonn Keogh and has a lot of research behind. The code in this notebook is mainly inspired in the XXVI paper:

- [IEEE - Matrix Profile XXVI: Mplots: Scaling Time Series Similarity Matrices to Massive Data](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10027730)

and following the guidelines of the example of use repository 

- [Zach Zimmerman - mplots-explorer (GitHub)](https://github.com/zpzim/mplot-explorer)

It also integrates methods of Stumpy:
- [Tutorial: The Matrix Profile](https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html#Time-Series-with-Length-n-=-13)

Feel free of modifying the parameters and open issue in case of error!

In [1]:
#| export
verbose                       = 1
reset_kernel                  = True
pre_configured_case           = True
case_id                       = 7
frequency_factor              = 1
frequency_factor_change_alias = True
cuda_device                   = 1

In [2]:
#| export
# This is only needed if the notebook is run in VSCode
import sys
import dvats.utils as ut
if '--vscode' in sys.argv:
    print("Executing inside vscode")
    ut.DisplayHandle.update = ut.update_patch

In [3]:
#| export
import dvats.config as cfg_

In [4]:
cfg_.show_available_configs()

Available datasets: 
0 - monash_australian_electricity_demand_0
1 - monash_solar_4_seconds_0
2 - wikipedia_0
3 - traffic_san_francisco_0
4 - monash_solar_10_minutes_0
5 - etth1_0
6 - stumpy_abp_0
7 - stumpy_toy_0


If you don't have the M-toy dataset. Please download it from [zenodo](https://zenodo.org/records/4328047/files/toy.csv). You can check more about it in [Stumpy](https://stumpy.readthedocs.io/en/latest/Tutorial_Multidimensional_Motif_Discovery.html) and see our previous analysis at the [Original DeepVATS paper](https://www.sciencedirect.com/science/article/pii/S0950705123005439).

## Main code
### Import libraries

## Main code
### Import libraries

In [5]:
#| export
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb


Octave is ready <oct2py.core.Oct2Py object at 0x7f540c9b2410>

















### Initialize and Configurate Artifact

#### Initialize Weights and Biases

In [6]:
#| export
wandb_api = wandb.Api()

#### Setup CUDA

In [7]:
#| export
device = torch.device(f'cuda:{cuda_device}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

gpu_device = torch.cuda.current_device()
print("GPU Device", gpu_device)
print("Device", device)
gpu_memory_status(gpu_device)

GPU Device 1
Device cuda:1
GPU | Used mem: 3
GPU | Used mem: 24
GPU | Memory Usage: [██------------------] 12%


### Get configutation from yml
> This file used the configuration files './config/base.yml' and './config/06_MPlots.ipynb'

In [8]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )

TypeError: force_artifact_config_mvp() got an unexpected keyword argument 'verbose'

### Setup Weights & biases artiffact

In [ ]:
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="06_MPlots"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

In [ ]:
job_type = 'MPlot'
job_type

### Start W&B Run

In [ ]:
#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity           = user,
    # work-nbs is a place to log draft runs
    project          = project,
    group            = config.wandb_group,
    job_type         = job_type,
    allow_val_change = True,
    mode             = config.analysis_mode,
    config           = config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous        = 'never' if config.use_wandb else 'must', 
    resume           = False,
    name             = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Generating the MPlots

#### Get W&B train artifact
Build artifact selector
Botch to use artifacts offline

In [ ]:
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

In [ ]:
#| export
df_train = train_artifact.to_df()

In [ ]:
#| export
if verbose > 0: 
    print(df_train.shape)
    display(df_train.head)
    print("df_train ~ ", df_train.shape)
    print("window_sizes = ", config.mvp_ws)
    print("wlen = ", config.w)
    df_train.head

In [ ]:
import dvats.mplots as mplots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.gridspec import GridSpec
import pyscamp as scamp
import dvats.memory as mem

In [ ]:
mem.gpu_memory_status()

In [ ]:
# Allows checking if pyscamp was built with CUDA and has GPU support.
has_gpu_support = scamp.gpu_supported()
has_gpu_support

In [ ]:
ts_step = 1
#the most relevant is T3
ts = df_train.iloc[:,2].values[::ts_step]
print(len(ts))

In [ ]:
#subsequence_len = config.w
subsequence_len = 30 #90 --> no sirve de nada
print(subsequence_len)

#### Compute MPlot

In [ ]:
data_MP = mplots.MatrixProfile(
    data      = ts, 
    data_b    = ts, 
    self_join = False
) 

In [ ]:
data_MP.provide_lens(1)
data_MP.dominant_lens

In [ ]:
data_MPlot = mplots.MatrixProfilePlot(
    DM_AB           = mplots.DistanceMatrix(), 
    MP_AB           = data_MP,
    data            = ts, 
    data_b          = ts,
    subsequence_len = subsequence_len,
    self_join       = False
)

In [ ]:
#| hide
print("MP_AB self_join", data_MPlot.MP_AB.self_join)
print(f"DM_AB ~ {data_MPlot.DM_AB.shape}")
print("MP_AB method:", data_MPlot.MP_AB.method)
print("DM_AB method:", data_MPlot.DM_AB.method)

In [ ]:
mplots.plot_with_dots(
    time_series = data_MPlot.data,
    title = "Checking expected motif", 
    fontsize = 10, 
    dots = False, 
    sequence_flag = False,
    figsize = (6,4)
)

In [ ]:
mp_method = 'stump'
dm_method = 'stump'

In [ ]:
# Threshold per method
threshold_scamp = 5
threshold_stump = 7.6
#Select the threshold for the method used for the computation
threshold = threshold_stump if data_MPlot.DM_AB.method == 'stump' else threshold_scamp

In [ ]:
data_MPlot.compute(
    mp_method           = mp_method, 
    dm_method           = dm_method,
    d                   = mplots.z_normalized_euclidean_distance,
    debug               = False,
    time_flag           = True,
    allow_experimental  = False,
    ensure_symetric     = False,
    max_points          = 10000,
    #nlens              = 5,
    subsequence_len     = subsequence_len,
    #provide_len        = True,
    provide_len         = False,
    downsample_flag     = True,
    min_lag             = 8,
    verbose             = 0,
    threads             = 1,
    gpus                = [1]
)

In [ ]:
n_a = len(data_MPlot.data)
n_b = len(data_MPlot.data_b)
print("-- DM Expected dimensions -- ")
print("Rows: ", n_b - data_MPlot.subsequence_len + 1)
print("Columns: ", n_a - data_MPlot.subsequence_len + 1)
print(f"-- Obtained shape when computing using  {data_MPlot.DM_AB.method} --")
print(data_MPlot.DM_AB.shape)
print(f"-- MP dimension using {data_MPlot.MP_AB.method}")
print(len(data_MPlot.MP_AB.distances))
print("-- DM Values --")
print(data_MPlot.DM_AB.distances)
print("-- MP values --")
print(data_MPlot.MP_AB.distances)

In [ ]:
data_MPlot.MP_AB.get_anomaly_idx()
print(data_MPlot.MP_AB.discord_idx)
print(data_MPlot.MP_AB.discord_nearest_neighbor_idx)
print(data_MPlot.MP_AB.discord_nearest_neighbor_idx_left)
print(data_MPlot.MP_AB.discord_nearest_neighbor_idx_right)

In [ ]:
%matplotlib widget
data_MPlot.MP_AB.plot_interactive(
    figsize = (8,8),
    verbose =  0
)

Looking for approximately this indexes

In [ ]:
mplots.plot_subsequence(
    TA = ts, 
    sequence_i = 30*5,
    subsequence_len = subsequence_len,
    dots = False,
    sequence_flag = False,
    save_plot = True,
    plot_path = "./images",
    subsequence_color = "orange",
    sequence_color = 'black'
)

In [ ]:
mplots.plot_subsequence(
    TA = ts, 
    sequence_i = 30*12,
    subsequence_len = subsequence_len,
    dots = False,
    sequence_flag = False,
    save_plot = True,
    plot_path = "./images",
    subsequence_color = "orange",
    sequence_color = "black"
)

what does the MPlot says?

In [ ]:
print("Threshold: ", threshold)
data_MPlot.plot(
    ts_name      = 'Variable 0',
    figsize      = (8,5),
    less_labels  = True,
    #r_min        = 0,
    #r_max        = 1000,
    #c_min        = 4500,
    #c_max        = 6000,
    dm_filter    = mplots.threshold_interval,
    th_min       = 8.5,
    th_max       = np.inf,
    include_min  = False,
    include_max  = True,
    gray_color   = False,
    plot_mp_flag = False,
    verbose      = 1
)

In [ ]:
print("Threshold: ", threshold)
data_MPlot.plot(
    ts_name      = 'Variable 0',
    figsize      = (8,5),
    less_labels  = True,
    r_min        = 0,
    r_max        = 150,
    c_min        = 50,
    c_max        = 200,
    dm_filter    = mplots.threshold_interval,
    th_min       = 7.8,
    th_max       = np.inf,
    include_min  = False,
    include_max  = True,
    gray_color   = False,
    plot_mp_flag = False,
    verbose      = 1
)

In [ ]:
print("Threshold: ", threshold)
data_MPlot.plot(
    ts_name      = 'Variable 0',
    figsize      = (8,5),
    less_labels  = True,
    r_min        = 0,
    r_max        = 300,
    c_min        = 250,
    c_max        = 550,
    dm_filter    = mplots.threshold_interval,
    th_min       = 7.5,
    th_max       = np.inf,
    include_min  = True,
    include_max  = True,
    gray_color   = False,
    plot_mp_flag = False,
    verbose      = 1
)